# Libraries

In [ ]:
!rm -rf model-outputs

In [ ]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install --upgrade pandas==1.2.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.9 MB 3.4 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5


In [ ]:
!pip install pandas-path fasttext pytorch-lightning --quiet

     |████████████████████████████████| 68 kB 2.4 MB/s 
     |████████████████████████████████| 584 kB 9.0 MB/s 
     |████████████████████████████████| 596 kB 30.1 MB/s 
     |████████████████████████████████| 409 kB 44.4 MB/s 
     |████████████████████████████████| 140 kB 37.4 MB/s 
     |████████████████████████████████| 1.1 MB 37.0 MB/s 
     |████████████████████████████████| 94 kB 2.3 MB/s 
     |████████████████████████████████| 144 kB 42.8 MB/s 
     |████████████████████████████████| 271 kB 46.8 MB/s 


In [ ]:
%matplotlib inline

import json
import logging
from pathlib import Path
import random
import tarfile
import tempfile
import warnings
from PIL import Image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import torchmetrics

import numpy as np
import pandas as pd
import pandas_path  # Path style access for pandas
from tqdm import tqdm


## Import FB AI Libraries
import torch                    
import torchvision
import fasttext

## PyTorch Lightning (for training)
import pytorch_lightning as pl


# Loading the Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/final_df1.csv')
data1

,Unnamed: 0,index,Filenames,img_vec,sent_bert,label
0,0,0,12285257.jpg,[6.98720098e-01 2.60966659e+00 3.61103010e+00 ...,"[0.35146206620000003, -0.11924376340000001, 1....",1
1,1,1,20612245.jpg,[1.02503991e+00 2.31855202e+00 5.80042601e-01 ...,"[-0.28953588010000003, 0.8174290061, 0.8659862...",1
2,2,2,24194267.jpg,[8.11383903e-01 1.91382349e+00 6.40732288e-01 ...,"[0.9257186055000001, 0.7144697905, 0.412784069...",1
3,3,3,10179556.jpg,[4.19537115e+00 1.97929633e+00 2.78581649e-01 ...,"[-0.0909568593, -0.0777853057, 1.7487983704, 0...",1
4,4,4,10181785.jpg,[1.84880745e+00 3.97921872e+00 4.33759719e-01 ...,"[0.27096313240000003, 1.0884246826, 0.81006759...",1
...,...,...,...,...,...,...
2201,987,987,132.jpg,[1.09574579e-01 1.17503452e+00 1.12803364e+00 ...,"[-0.1283777803, 0.4022460282, 1.6771719456, 0....",0
2202,988,988,765.jpg,[1.043156 1.8925457 3.2395842 0.92683613 0...,"[-0.0911655799, 0.0332999937, 2.4850916862, 0....",0
2203,989,989,107.jpg,[2.85617650e-01 6.63005412e-01 3.22972164e-02 ...,"[0.26998162270000003, 0.0166379828, 0.88493543...",0
2204,990,990,666.jpg,[2.38225177e-01 1.68061122e-01 1.35491371e+00 ...,"[0.5512937307, 0.0641091838, 0.6356454492, 0.0...",0


In [ ]:
data1['img_vec'] = data1['img_vec'].apply(lambda x: str(x.strip('[').strip(']')))
data1

,Unnamed: 0,index,Filenames,img_vec,sent_bert,label
0,0,0,12285257.jpg,6.98720098e-01 2.60966659e+00 3.61103010e+00 1...,"[0.35146206620000003, -0.11924376340000001, 1....",1
1,1,1,20612245.jpg,1.02503991e+00 2.31855202e+00 5.80042601e-01 4...,"[-0.28953588010000003, 0.8174290061, 0.8659862...",1
2,2,2,24194267.jpg,8.11383903e-01 1.91382349e+00 6.40732288e-01 5...,"[0.9257186055000001, 0.7144697905, 0.412784069...",1
3,3,3,10179556.jpg,4.19537115e+00 1.97929633e+00 2.78581649e-01 1...,"[-0.0909568593, -0.0777853057, 1.7487983704, 0...",1
4,4,4,10181785.jpg,1.84880745e+00 3.97921872e+00 4.33759719e-01 1...,"[0.27096313240000003, 1.0884246826, 0.81006759...",1
...,...,...,...,...,...,...
2201,987,987,132.jpg,1.09574579e-01 1.17503452e+00 1.12803364e+00 5...,"[-0.1283777803, 0.4022460282, 1.6771719456, 0....",0
2202,988,988,765.jpg,1.043156 1.8925457 3.2395842 0.92683613 0....,"[-0.0911655799, 0.0332999937, 2.4850916862, 0....",0
2203,989,989,107.jpg,2.85617650e-01 6.63005412e-01 3.22972164e-02 1...,"[0.26998162270000003, 0.0166379828, 0.88493543...",0
2204,990,990,666.jpg,2.38225177e-01 1.68061122e-01 1.35491371e+00 1...,"[0.5512937307, 0.0641091838, 0.6356454492, 0.0...",0


In [ ]:
data1['img_vec'] = data1['img_vec'].apply(lambda x: str(x.split()))

In [ ]:
import ast
def tonums(x):
  return ast.literal_eval(x)

In [ ]:
def to_nums(x): 
  return [float(i) for i in x]

In [ ]:
data1['img_vec'] = data1['img_vec'].apply(lambda x: tonums(x))
data1['sent_bert'] = data1['sent_bert'].apply(lambda x: tonums(x))
# random_data['img_vec'] = ['img_vec'].apply(lambda x: tonums(x))
# new_merge['sent_bert'] = new_merge['sent_bert'].apply(lambda x: tonums(x))

In [ ]:
data1['img_vec'] = data1['img_vec'].apply(lambda x: to_nums(x))

In [ ]:
data1['label'] = data1['label'].apply(lambda x: int(x))

In [ ]:
data2 = data1.sample(frac=0.1)

In [ ]:
train_input, dev_input = train_test_split(data1, random_state = 42, train_size=0.8)
dev_input, test_input = train_test_split(dev_input, random_state=42, train_size=0.5)

In [ ]:
len(test_input)

111

# MultiModal Model

In [ ]:
class MemesDataset(torch.utils.data.Dataset):
    def __init__(
      self,
      data,
      image_vector,
      text_vector,
      balance=False,
      dev_limit=None,
      random_state=0,
  ):

      self.samples_frame = data
      self.image_transform = torch.tensor(image_vector.tolist())
      self.text_transform = torch.tensor(text_vector.tolist())

    def __len__(self):
        """This method is called when you do len(instance)
            for an instance of this class.
            """
        return len(self.samples_frame)

    def __getitem__(self, idx):
        """This method is called when you do instance[key]
            for an instance of this class.
            """
        if torch.is_tensor(idx):
            idx = idx.tolist()


        img_id = self.samples_frame.loc[idx, "Filenames"]

        image = self.image_transform
        text = self.text_transform

        if "label" in self.samples_frame.columns:
            label = torch.Tensor([self.samples_frame.loc[idx,
                                                       "label"]]).long().squeeze()
            sample = {"id": img_id, "image": image, "text": text, "label": label}
        else:
            sample = {"id": img_id, "image": image, "text": text}

        return sample


In [ ]:
class TextAndImageConcat(torch.nn.Module):

    def __init__(
      self,
      num_classes,
      loss_fn,
      language_module,
      vision_module,
      language_feature_dim,
      vision_feature_dim,
      fusion_output_size,
      dropout_p,
  ):
        super(TextAndImageConcat, self).__init__()
        self.language_module = language_module
        self.vision_module = vision_module
        self.fusion = torch.nn.Linear(in_features=(language_feature_dim +
                                                   vision_feature_dim),
                                      out_features=fusion_output_size)
        self.fc = torch.nn.Linear(in_features=fusion_output_size,
                                  out_features=num_classes)
        self.loss_fn = loss_fn
        self.dropout = torch.nn.Dropout(dropout_p)

    def forward(self, text, image, label=None):
        if image.shape[1] != 3: 
          if image.shape[1] == 2:
            pd = (0, 0, 0, 0, 0, 1)
            image = torch.nn.functional.pad(image, pd, 'constant', 0)
            text = torch.nn.functional.pad(text, pd, 'constant', 0)
          elif image.shape[1] == 1: 
            pd = (0, 0, 0, 0, 0, 2)
            image = torch.nn.functional.pad(image, pd, 'constant', 0)
            text = torch.nn.functional.pad(text, pd, 'constant', 0)
        
        text_features = torch.nn.functional.relu(self.language_module(text))
        try:
          image_features = torch.nn.functional.relu(self.vision_module(image))
          image_features = image_features.unsqueeze(1)
        except: 
          if image.shape[0] == 1:
            pd = (0, 0, 0, 0, 0, 2)
            image = torch.nn.functional.pad(image, pd, 'constant', 0)
          elif image.shape[0] == 2: 
            pd = (0, 0, 0, 0, 0, 1)
            image = torch.nn.functional.pad(image, pd, 'constant', 0)
          image = image.unsqueeze(0)
          image_features = torch.nn.functional.relu(self.vision_module(image))
          image_features = image_features.unsqueeze(1)
        p3d_text = (256, 256, 0, 0 , 0, 0)
        p3d_image = (9712, 9712, 0, 0, 1, 1)
        image_features = torch.nn.functional.pad(image_features, p3d_image, 'constant', 0)
        text_features = torch.nn.functional.pad(text_features, p3d_text, 'constant', 0)
        try:
          combined = torch.cat([text_features, image_features], dim=1)
        except Exception: 
          pass
        try:
          pd = (0, 0, 0, 0, 0, 1)
          text_features = torch.nn.functional.pad(text_features, pd, 'constant', 0)
          combined = torch.cat([text_features, image_features], dim=1)
        except Exception: 
          pass
        try: 
          pd = (0, 0, 0, 0, 1, 1)
          text_features = torch.nn.functional.pad(text_features, pd, 'constant', 0)
          combined = torch.cat([text_features, image_features], dim=1)
        except Exception:
          pass
        fused = self.dropout(torch.nn.functional.relu(self.fusion(combined)))
        logits = self.fc(fused)
        pred = torch.nn.functional.softmax(logits)
        if label != None:
          label = label.unsqueeze(1)
          p2d = (1, 0, 0, 0)
          label = torch.nn.functional.pad(label, p2d, 'constant', 0)
          if label.shape[0] != pred.shape[0]: 
            if label.shape[0] == 2:
              p = (0, 0, 0, 1)
              label = torch.nn.functional.pad(label, p, 'constant', 0)
            elif label.shape[0] ==1: 
              p = (0, 0, 1, 1)
              label = torch.nn.functional.pad(label, p, 'constant', 0)
          loss = (self.loss_fn(pred, label) if label is not None else label)
          return (pred, loss)
        else:
          return (pred, 1)


# Training

In [ ]:
class HatefulMemesModel(pl.LightningModule):
  def __init__(self, hparams):

    super(HatefulMemesModel, self).__init__()
    self.save_hyperparameters(hparams)
      # assign some hparams that get used in multiple places
    self.valid_acc = torchmetrics.Accuracy()
    self.embedding_dim = self.hparams.get("embedding_dim", 300)
    self.language_feature_dim = self.hparams.get("language_feature_dim", 768)
    self.vision_feature_dim = self.hparams.get(
        # balance language and vision features by default
        "vision_feature_dim",
        self.language_feature_dim)
    self.output_path = Path(self.hparams.get("output_path", "model-outputs"))
    self.output_path.mkdir(exist_ok=True)

    # instantiate transforms, datasets
    self.text_transform = self.hparams.get("data")['sent_bert']
    self.image_transform = self.hparams.get("data")['img_vec']
    self.train_dataset = self._build_dataset(self.hparams.get("train_path"))
    self.dev_dataset = self._build_dataset(self.hparams.get("dev_path"))

    # set up model and training
    self.model = self._build_model()
    self.trainer_params = self._get_trainer_params()

## Required LightningModule Methods (when validating) ##

  def forward(self, text, image, label=None):
      image = image.unsqueeze(0)
      return self.model(text, image, label)

  def training_step(self, batch, batch_nb):
      preds, loss = self.forward(text=batch["text"],
                              image=batch["image"],
                              label=batch["label"])

      return {"loss": loss}

  def validation_step(self, batch, batch_nb):
      preds, loss = self.eval().forward(text=batch["text"],
                                    image=batch["image"],
                                    label=batch["label"])
      if batch['label'].shape != preds[:,0].argmax(axis=1).shape: 
        pd = (1, 1)
        batch['label'] = torch.nn.functional.pad(batch['label'], pd, 'constant', 0)
      self.valid_acc.update(preds[:,0].argmax(axis=1), batch['label'])
      self.log('valid_acc', self.valid_acc, on_step=True, on_epoch=True)
      return {"batch_val_loss": loss, 'valid_acc': self.valid_acc}

  def validation_epoch_end(self, outputs):
      avg_loss = torch.stack(tuple(
          output["batch_val_loss"] for output in outputs)).mean()

      return {"val_loss": avg_loss, "progress_bar": {"avg_val_loss": avg_loss} 
              }


  def configure_optimizers(self):
      optimizers = [
          torch.optim.AdamW(self.model.parameters(),
                        lr=self.hparams.get("lr", 0.001))
      ]
  
      return optimizers

  def train_dataloader(self):
      return torch.utils.data.DataLoader(
          self.train_dataset,
          shuffle=True,
          batch_size=self.hparams.get("batch_size", 3),
          num_workers=self.hparams.get("num_workers", 2))

  def val_dataloader(self):
      return torch.utils.data.DataLoader(
          self.dev_dataset,
          shuffle=False,
          batch_size=self.hparams.get("batch_size", 3),
          num_workers=self.hparams.get("num_workers", 2))

  def fit(self):
      self._set_seed(self.hparams.get("random_state", 42))
      self.trainer = pl.Trainer(**self.trainer_params)
      self.trainer.fit(self)

  def _set_seed(self, seed):
      random.seed(seed)
      np.random.seed(seed)
      torch.manual_seed(seed)
      if torch.cuda.is_available():
          torch.cuda.manual_seed_all(seed)

  def _build_dataset(self, dataset_key):
      return MemesDataset(
      self.hparams.get("data"),
      image_vector=self.image_transform,
      text_vector=self.text_transform)

  def _build_model(self):
    language_module = torch.nn.Linear(in_features=self.embedding_dim,
                                      out_features=self.language_feature_dim)

    vision_module = torchvision.models.resnet152(pretrained=True)
    vision_module.fc = torch.nn.Linear(in_features=2048,
                                        out_features=self.vision_feature_dim)

    return TextAndImageConcat(
        num_classes=self.hparams.get("num_classes", 2),
        loss_fn=torch.nn.CrossEntropyLoss(),
        language_module=language_module,
        vision_module=vision_module,
        language_feature_dim=self.language_feature_dim,
        vision_feature_dim=self.vision_feature_dim,
        fusion_output_size=self.hparams.get("fusion_output_size", 512),
        dropout_p=self.hparams.get("dropout_p", 0.1),
    )

  def _get_trainer_params(self):
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        dirpath=self.output_path,
        monitor=self.hparams.get("checkpoint_monitor", "valid_acc_epoch"),
        mode=self.hparams.get("checkpoint_monitor_mode", "min"),
        verbose=self.hparams.get("verbose", True))

    early_stop_callback = pl.callbacks.EarlyStopping(
        monitor=self.hparams.get("early_stop_monitor", "valid_acc_epoch"),
        min_delta=self.hparams.get("early_stop_min_delta", 0.001),
        patience=self.hparams.get("early_stop_patience", 3),
        verbose=self.hparams.get("verbose", True),
    )

    trainer_params = {
            "callbacks": [early_stop_callback, checkpoint_callback],
            "weights_save_path": self.output_path,
            "accumulate_grad_batches": self.hparams.get(
                "accumulate_grad_batches", 1
            ),
            "gpus": self.hparams.get("n_gpu", 1),
            "max_epochs": self.hparams.get("max_epochs", 100),
            "gradient_clip_val": self.hparams.get(
                "gradient_clip_value", 1
            ),
        }
    return trainer_params
  
  def make_submission_frame(self, test_path):
      test_dataset = self._build_dataset(test_path)
      submission_frame = pd.DataFrame(index=test_dataset.samples_frame.Filenames,
                                      columns=["proba", "label"])
      test_dataloader = torch.utils.data.DataLoader(
          test_dataset,
          shuffle=False,
          batch_size=self.hparams.get("batch_size", 3),
          num_workers=self.hparams.get("num_workers", 2))
      for batch in tqdm(test_dataloader, total=len(test_dataloader)):
          preds, _ = self.model.eval().to("cpu")(batch["text"], batch["image"])
          preds = preds.detach().numpy()
          try:
            submission_frame.loc[batch["id"], "proba"] = preds[:, 1][:, 0]
            submission_frame.loc[batch["id"], "label"] = preds[:,0].argmax(axis=1)
          except: 
            submission_frame.loc[batch["id"], "proba"] = preds[:, 1][:, 0][1:]
            submission_frame.loc[batch["id"], "label"] = preds[:,0][1:].argmax(axis=1)
      submission_frame.proba = submission_frame.proba.astype(float)
      submission_frame.label = submission_frame.label.astype(int)
      return submission_frame


In [ ]:
hparams = {
    
    # Required hparams
    "data": data1.reset_index(),
    "train_path": train_input.reset_index(),
    "dev_path": dev_input.reset_index(),
    # Optional hparams
    "embedding_dim": 768,
    "language_feature_dim": 19424,
    "vision_feature_dim": 512,
    "fusion_output_size": 3645,
    "output_path": "model-outputs",
    "dev_limit": None,
    "lr": 0.00005,
    "max_epochs": 10,
    "n_gpu": 1,
    "batch_size": 3,
    # allows us to "simulate" having larger batches 
    "accumulate_grad_batches": 16,
    "early_stop_patience": 3,
}


In [ ]:
hateful_memes_model1 = HatefulMemesModel(hparams=hparams)
hateful_memes_model1.fit()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:58: LightningDeprecationWarning: Setting `Trainer(weights_save_path=)` has been deprecated in v1.6 and will be removed in v1.8. Please pass ``dirpath`` directly to the `ModelCheckpoint` callback
  "Setting `Trainer(weights_save_path=)` has been deprecated in v1.6 and will be"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning: Checkpoint directory /content/model-outputs exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params
-------------------------------------------------
0 | valid_acc | Accuracy           | 0     
1 | model     | TextAndImageConcat | 146 M 
----

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:72: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


...

torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Si

Epoch 2, global step 138: 'valid_acc_epoch' was not in top 1


Validation: 0it [00:00, ?it/s]

torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Size([3]) torch.Size([3])
torch.Si

Monitored metric valid_acc_epoch did not improve in the last 3 records. Best score: 0.450. Signaling Trainer to stop.
Epoch 3, global step 184: 'valid_acc_epoch' was not in top 1


# Submission and Results

In [ ]:
# we should only have saved the best checkpoint
checkpoints = list(Path("model-outputs").glob("*.ckpt"))
assert len(checkpoints) == 1

checkpoints

[PosixPath('model-outputs/epoch=0-step=5.ckpt')]

In [ ]:
checkpoint = Path('/content/drive/My Drive/Colab Notebooks/model-outputs2/epoch=0-step=46.ckpt')

In [ ]:
hateful_memes_model = HatefulMemesModel.load_from_checkpoint(str(checkpoints[0]))
submission = hateful_memes_model.make_submission_frame(test_input.iloc[0])
submission.head()

  0%|          | 0/74 [00:00<?, ?it/s]

torch.Size([3, 221, 512])
torch.Size([4, 221, 19936]) torch.Size([3, 1, 19936])
torch.Size([6, 221, 19936]) torch.Size([3, 1, 19936])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  1%|▏         | 1/74 [00:02<03:30,  2.88s/it]

torch.Size([3, 221, 512])
torch.Size([4, 221, 19936]) torch.Size([3, 1, 19936])
torch.Size([6, 221, 19936]) torch.Size([3, 1, 19936])


  3%|▎         | 2/74 [00:04<02:34,  2.14s/it]

...

torch.Size([3, 221, 512])
torch.Size([4, 221, 19936]) torch.Size([3, 1, 19936])
torch.Size([6, 221, 19936]) torch.Size([3, 1, 19936])


 99%|█████████▊| 73/74 [01:58<00:01,  1.58s/it]

torch.Size([2, 221, 512])
torch.Size([2, 221, 19936]) torch.Size([3, 1, 19936])
torch.Size([5, 221, 19936]) torch.Size([3, 1, 19936])


100%|██████████| 74/74 [01:59<00:00,  1.62s/it]


,proba,label
Filenames,,
11776255.jpg,0.333333,0
943.jpg,0.333333,0
11999399.jpg,0.333333,0
11227148.jpg,0.333333,0
11379425.jpg,0.333333,0


In [ ]:
submission = hateful_memes_model.make_submission_frame(test_input)
submission.head()

  0%|          | 0/74 [00:00<?, ?it/s]

torch.Size([3, 221, 512])
torch.Size([4, 221, 19936]) torch.Size([3, 1, 19936])
torch.Size([6, 221, 19936]) torch.Size([3, 1, 19936])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  1%|▏         | 1/74 [00:06<07:19,  6.03s/it]

torch.Size([3, 221, 512])
torch.Size([4, 221, 19936]) torch.Size([3, 1, 19936])
torch.Size([6, 221, 19936]) torch.Size([3, 1, 19936])


  3%|▎         | 2/74 [00:08<05:00,  4.18s/it]

...

torch.Size([3, 221, 512])
torch.Size([4, 221, 19936]) torch.Size([3, 1, 19936])
torch.Size([6, 221, 19936]) torch.Size([3, 1, 19936])


 99%|█████████▊| 73/74 [02:03<00:01,  1.59s/it]

torch.Size([2, 221, 512])
torch.Size([2, 221, 19936]) torch.Size([3, 1, 19936])
torch.Size([5, 221, 19936]) torch.Size([3, 1, 19936])


100%|██████████| 74/74 [02:05<00:00,  1.70s/it]


,proba,label
Filenames,,
11776255.jpg,0.333333,0
943.jpg,0.333333,0
11999399.jpg,0.333333,0
11227148.jpg,0.333333,0
11379425.jpg,0.333333,0
